In [ ]:
import os
import pickle
from utils import get_spike_counts_over_time_windows
import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
filename = '150945172328ree42_dur100_brain1.p'

folder = '/Users/Jan/Dropbox/Master/mackelab/code/balanced_clustered_network/data/'

fullname = os.path.join(folder, filename)

with open(fullname, 'rb') as handle:
    round_dict = pickle.load(handle)

data = round_dict['trial0']
params = round_dict['params']

smi = data['spikes_I']
sme = data['spikes_E']

In [ ]:
# calculate population firing rate mean
window_length = 0.01  # 10ms
start_t = 1.
delta_t = 10.


In [ ]:
features = []
corrs0 = []
corrs10 = []
corrs20 = []
counts = []

for spike_times in [sme, smi]:
    spike_counts = get_spike_counts_over_time_windows(spike_times, start_t, delta_t, window_length)
    spike_counts = spike_counts.mean(axis=0)
    counts.append(spike_counts)

    # take mean across bins to get the overall average firing rate as spikes / 10ms
    features.append(spike_counts.mean())

    # 0-lag covariance taken as std across windows
    zero_lag_std = np.std(spike_counts)
    corrs0.append(zero_lag_std)

In [ ]:
counts = np.array(counts).T
counts.shape

In [ ]:
# 0 lag cross correlation of spike counts
corrs0.append(np.corrcoef(counts[:,0], counts[:, 1])[0, 1])

In [ ]:
# one bin is 10ms 
k = 1
# shift by lag
shift = counts[k:,:]
orig = counts[:-k,:]
# put all in one mat with rows: [Es, Is, Eo, Io]
xy = np.array(np.concatenate([shift, orig],axis=1).T)

# we are interested in EsEo, EsIo, IsEo, IsIo --> the upper right 2d matrix 
sub_mat = np.corrcoef(xy,rowvar=1)[:2, 2:]
corrs10 = list(sub_mat.flatten())

In [ ]:
# one bin is 10ms 
k = 2
# shift by lag
shift = counts[k:,:]
orig = counts[:-k,:]
# put all in one mat with rows: [Es, Is, Eo, Io]
xy = np.array(np.concatenate([shift, orig],axis=1).T)

# we are interested in EsEo, EsIo, IsEo, IsIo --> the upper right 2d matrix 
sub_mat = np.corrcoef(xy,rowvar=1)[:2, 2:]
corrs20 = list(sub_mat.flatten())

In [ ]:
features += corrs0 + corrs10 + corrs20

In [ ]:
features